In [7]:
import pandas as pd 
import kang
from tqdm import tqdm

In [2]:
ingff  =  './pregff.gff3'
df_gff = pd.read_csv(ingff,comment='#',sep='\t',header=None)

In [4]:
df_gff[2].unique()

array(['gene', 'mRNA', 'CDS', 'three_prime_UTR', 'five_prime_UTR'],
      dtype=object)

In [13]:
startnum = 0
dic      = {}
dic_t    = {} # transcript count 
dic_c    = {} # cds count 
dic_5u   = {}
dic_3u   = {}
new_col  = []
for ix in tqdm(df_gff.index):
    edf  = df_gff.loc[ix]
    scaf = edf[0] 
    if edf[2] == 'gene': ## ID=g1
        geneID = scaf+'_'+kang.infoparse(edf[8])['ID']
        newgeneID = 'Waus{:05d}g'.format(int(scaf.replace('scaffold',''))) + '{:05d}'.format(startnum)
        dic[geneID] = newgeneID 
        new_col.append('ID='+newgeneID)
        startnum += 1
    elif edf[2] == 'mRNA': ## ID=g1.t1;Parent=g1
        geneID          = scaf+'_'+kang.infoparse(edf[8])['Parent']
        newgeneID       = dic[geneID]
        try:
            dic_t[newgeneID] += 1
        except KeyError:
            dic_t[newgeneID] = 1
        transcriptID    = scaf+'_'+kang.infoparse(edf[8])['ID']
        #newtranscriptID = newgeneID+'.'+transcriptID.split('.')[1]
        newtranscriptID = newgeneID+'.'+str(dic_t[newgeneID])
        dic[transcriptID] = newtranscriptID 
        new_col.append('ID='+newtranscriptID+';'+'Parent='+newgeneID)
    elif edf[2] == 'CDS': ## ID=g1.t1.cds;Parent=g1.t1
        transcriptID       = scaf+'_'+kang.infoparse(edf[8])['Parent']
        newtranscriptID    = dic[transcriptID]
        try:
            dic_c[newtranscriptID] += 1
        except KeyError:
            dic_c[newtranscriptID] = 1
        cdsID              = scaf+'_'+kang.infoparse(edf[8])['ID']
        newcdsID           = newtranscriptID+'.'+str(dic_c[newtranscriptID])
        new_col.append('ID=cds.'+newcdsID+';'+'Parent='+newtranscriptID)
    elif edf[2] == 'three_prime_UTR': ## ID=g1.t1.cds;Parent=g1.t1
        transcriptID       = scaf+'_'+kang.infoparse(edf[8])['Parent']
        newtranscriptID    = dic[transcriptID]
        try:
            dic_3u[newtranscriptID] += 1
        except KeyError:
            dic_3u[newtranscriptID] = 1
        cdsID              = scaf+'_'+kang.infoparse(edf[8])['ID']
        newcdsID           = newtranscriptID+'.'+str(dic_3u[newtranscriptID])
        new_col.append('ID=3utr.'+newcdsID+';'+'Parent='+newtranscriptID)
    elif edf[2] == 'five_prime_UTR': ## ID=g1.t1.cds;Parent=g1.t1
        transcriptID       = scaf+'_'+kang.infoparse(edf[8])['Parent']
        newtranscriptID    = dic[transcriptID]
        try:
            dic_5u[newtranscriptID] += 1
        except KeyError:
            dic_5u[newtranscriptID] = 1
        cdsID              = scaf+'_'+kang.infoparse(edf[8])['ID']
        newcdsID           = newtranscriptID+'.'+str(dic_5u[newtranscriptID])
        new_col.append('ID=5utr.'+newcdsID+';'+'Parent='+newtranscriptID)

100%|██████████| 291123/291123 [02:21<00:00, 2050.59it/s]


In [14]:
#m = df_gff[2].isin(['gene','mRNA','CDS'])
df_gff_sub = df_gff
df_gff_sub[9] = new_col
df_gff_sub[[0,1,2,3,4,5,6,7,9]].to_csv(ingff+'.corrected.gff3',sep='\t',index=False,header=None)